# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [2]:
# YOUR CHANGES HERE
import pandas as pd

df = pd.read_csv("C:/Users/eleat/OneDrive/Documents/OMDS Last Semester/strawberry-prices.tsv", sep='\t')
df['month'] = pd.to_datetime(df['month'])

train_data = df[df["month"].dt.year <= 2023].copy()
train_data["month_numbers"] = train_data["month"].dt.month

all_year_avg = (train_data.groupby("month_numbers")['price'].mean().reset_index())

months_2024 = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')

forecast = []
for date in months_2024:
    month_numbers = date.month
    price = all_year_avg.loc[all_year_avg['month_numbers'] == month_numbers, 'price'].values[0]
    forecast.append([date.strftime('%Y-%m-01'), price])
forecast_df = pd.DataFrame(forecast, columns=['month', 'price'])

forecast_df.to_csv('strawberry-backtest.tsv', sep='\t', index=False)

forecast_df

,month,price
0,2024-01-01,4.362750
1,2024-02-01,4.091250
2,2024-03-01,3.688750
3,2024-04-01,3.787200
4,2024-05-01,3.526250
5,2024-06-01,3.275000
6,2024-07-01,3.194000
7,2024-08-01,3.489750
8,2024-09-01,3.581750
9,2024-10-01,3.926000


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [3]:
# YOUR CHANGES HERE

df = pd.read_csv("C:/Users/eleat/OneDrive/Documents/OMDS Last Semester/strawberry-prices.tsv", sep='\t')
df['month'] = pd.to_datetime(df['month'])

true_2024 = df[df["month"].dt.year == 2024][['month', 'price']].copy()
true_2024= true_2024.rename(columns={'price': 'true_price'})

pred_2024 = pd.read_csv('strawberry-backtest.tsv', sep='\t')
pred_2024['month'] = pd.to_datetime(pred_2024['month'])
pred_2024 = pred_2024.rename(columns={'price': 'predicted_price'})

merged = true_2024.merge(pred_2024, on='month', how='inner')
merged['residual'] = (merged['true_price'] - merged['predicted_price'])

backtest = pd.DataFrame({'mean': [merged['residual'].mean()],'std': [merged['residual'].std()]})

backtest.to_csv('backtest-accuracy.tsv', sep='\t', index=False)

Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [4]:
# YOUR CHANGES HERE

df = pd.read_csv("C:/Users/eleat/OneDrive/Documents/OMDS Last Semester/strawberry-prices.tsv", sep='\t')
df['month'] = pd.to_datetime(df['month'])

train_data = df[df["month"].dt.year <= 2024].copy()
train_data["month_numbers"] = train_data["month"].dt.month

monthly_forecast = (train_data.groupby("month_numbers")['price'].mean().reset_index())

months_2025 = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')

forecast = []
for date in months_2025:
    month_numbers = date.month
    price = monthly_forecast.loc[monthly_forecast['month_numbers'] == month_numbers, 'price'].values[0]
    forecast.append([date.strftime('%Y-%m-01'), price])

forecast_df = pd.DataFrame(forecast, columns=['month', 'price'])
forecast_df.to_csv('strawberry-forecast.tsv', sep='\t', index=False)

forecast_df

,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [5]:
# YOUR CHANGES HERE
import numpy as np

df = pd.read_csv("strawberry-forecast.tsv", sep='\t')
df['month'] = pd.to_datetime(df['month'])
df = df.sort_values('month').reset_index(drop=True)

investment = 1000000
freeze_cost = 0.20
store_cost = 0.10
discount = 0.10

rows = []
for i in range(len(df)):
    buy_month = df.loc[i, 'month']
    buy_price = df.loc[i, 'price']

    for j in range(i + 1, len(df)):
        sell_month = df.loc[j, 'month']
        sell_price = df.loc[j, 'price']

        months = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)
        cost_pint = buy_price + freeze_cost + store_cost * months
        # Discount applied to selling price with 10% reduction
        income_pint = 0.9 * sell_price
        profit_pint = income_pint - cost_pint

        pints_purchased = np.floor(investment / cost_pint)
        expected_profit = profit_pint * pints_purchased

        rows.append([buy_month.strftime('%Y-%m-01'), sell_month.strftime('%Y-%m-01'), pints_purchased, expected_profit])

results = pd.DataFrame(rows, columns=['buy_month', 'sell_month', 'pints_purchased', 'expected_profit'])
results.to_csv('timings.tsv', sep='\t', index=False)

results

,buy_month,sell_month,pints_purchased,expected_profit
0,2025-01-01,2025-02-01,208281.0,-226605.562380
1,2025-01-01,2025-03-01,204031.0,-320683.683940
2,2025-01-01,2025-04-01,199952.0,-326068.924672
3,2025-01-01,2025-05-01,196032.0,-388072.788480
4,2025-01-01,2025-06-01,192263.0,-443616.110420
...,...,...,...,...
61,2025-09-01,2025-11-01,249140.0,-12865.589600
62,2025-09-01,2025-12-01,243084.0,50618.353656
63,2025-10-01,2025-11-01,238982.0,-53111.359680
64,2025-10-01,2025-12-01,233404.0,8783.926136


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [9]:
# YOUR CHANGES HERE

results = pd.read_csv("timings.tsv", sep='\t')
best_row = results['expected_profit'].idxmax()
best_profit = results.loc[best_row, 'expected_profit']
pints_purchased = results.loc[best_row, 'pints_purchased']

part_2 = pd.read_csv("backtest-accuracy.tsv", sep='\t')
std_residual = part_2.loc[0, 'std']
one_std_profit = pints_purchased * std_residual

check = pd.DataFrame({'best_profit': [best_profit],'one_std_profit': [one_std_profit]})

check.to_csv('check.tsv', sep='\t', index=False)
check

,best_profit,one_std_profit
0,114385.703292,70793.566481


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [7]:
# I did use Gemini for part 1 to help me gain a better understaing of prediciton accuracy. I was torn between just using 2023 data for the most recent prediciton, but I realize I may not be covering the whole story. 
# Here is the link for the Gemini chat I used: https://gemini.google.com/share/b9dc9e7295e9
# For part 4, I did use Gemini AI to help me a command to use the whole pints rather than a partial pint when calculating profit. I forgot the command. https://gemini.google.com/share/e43df78ba62d

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.